In [18]:
!pip install kaggle --quiet
!pip install pandas --quiet
!pip install datasets transformers --quiet

You should consider upgrading via the 'D:\LakeheadUCourse\3rd_year_winter\BigData_COMP4311\suicideRate\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'D:\LakeheadUCourse\3rd_year_winter\BigData_COMP4311\suicideRate\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'D:\LakeheadUCourse\3rd_year_winter\BigData_COMP4311\suicideRate\venv\Scripts\python.exe -m pip install --upgrade pip' command.


## Download Dataset

In [2]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Specify the directory where you want to store the dataset
dataset_directory = os.path.join(os.getcwd(), "data")  # This will create a 'data' folder in the current working directory

# Kaggle API authentication
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset_name = 'aunanya875/suicidal-tweet-detection-dataset'
dataset_path = os.path.join(dataset_directory, api.dataset_list_files(dataset_name).files[0].name)

if not os.path.exists(dataset_path):
    api.dataset_download_files(dataset_name, path=dataset_directory, unzip=True)
    print("Dataset downloaded and extracted to:", dataset_directory)

Dataset downloaded and extracted to: D:\LakeheadUCourse\3rd_year_winter\BigData_COMP4311\suicideRate\data


In [3]:
import pandas as pd

df = pd.read_csv(dataset_path)
df.rename(columns={'Suicide': 'label', 'Tweet': 'text'}, inplace=True)

unique_suicide_values = df['label'].unique()
unique_suicide_values

array(['Not Suicide post', 'Potential Suicide post '], dtype=object)

In [4]:
df['label'] = df['label'].astype('category').cat.codes.astype(float)
df.head()

,text,label
0,making some lunch,0.0
1,@Alexia You want his money.,0.0
2,@dizzyhrvy that crap took me forever to put to...,1.0
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,0.0
4,Trying out &quot;Delicious Library 2&quot; wit...,0.0


## Clean Up Dataset

In [5]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

def preprocess_text(text):
    # Lowercase the text
    text = str(text).lower()

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]

    # Stem the tokens
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    return " ".join(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\weikang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df['text'] = df['text'].apply(preprocess_text)

df['text'].head()

0                                           make lunch
1                                    alexia want money
2    dizzyhrvi crap took forev put togeth iâm go ...
3         jnaylor kiwitweet hey jer sinc start twitter
4    tri quotdelici librari 2quot mix result bar co...
Name: text, dtype: object

In [7]:
import transformers
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

D:\LakeheadUCourse\3rd_year_winter\BigData_COMP4311\suicideRate\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset = Dataset.from_pandas(df,preserve_index=False) 
dataset = dataset.train_test_split(test_size=0.3) 
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1250
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 537
    })
})

In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 537/537 [00:00<00:00, 2931.94 examples/s]


In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1250
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 537
    })
})

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

## Model Training

In [14]:
!pip install transformers[torch]
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

You should consider upgrading via the 'D:\LakeheadUCourse\3rd_year_winter\BigData_COMP4311\suicideRate\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://download.pytorch.org/whl/nightly/cu121


You should consider upgrading via the 'D:\LakeheadUCourse\3rd_year_winter\BigData_COMP4311\suicideRate\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [15]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="suicide_rate_model_out",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=10,
                                  save_total_limit = 2,
                                  save_strategy = 'epoch',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the trained model from a local directory
model_path = "suicide_rate_model_out"  # Replace with your model path
model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)

# Input text
input_text = "This is a test sentence."

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True)

# Make predictions
with torch.no_grad():
    logits = model(**inputs).logits

# Convert logits to predicted values (assuming a single regression output)
predicted_value = logits.item()

# Print results
print("Input Text:", input_text)
print("Predicted Value:", predicted_value)